In [184]:
from scipy.io import loadmat
mat = loadmat('test.mat')
audio_data=mat['audio_data']
audio_Fs=1000

In [185]:
#now run through each function to compare with the output with matlab

In [186]:
def butterworth_low_pass_filter(original_signal,order,cutoff,sampling_frequency, figure=False):
    from scipy import signal
    b, a = signal.butter(order, 2*cutoff/sampling_frequency, 'low')
    print(np.squeeze(original_signal).shape)
    low_pass_filtered_signal = signal.filtfilt(b, a, np.squeeze(original_signal),padlen=3*max(len(a), len(b))-3)
    return low_pass_filtered_signal

In [187]:
mat=loadmat('audio_lowpass.mat')
audio_lowpassmat=mat['audio_data_lowpass']

In [188]:
print(audio_lowpassmat)

[[ 0.06471533]
 [ 0.42868168]
 [ 0.31707522]
 ...
 [-0.06511216]
 [-0.13562029]
 [-0.1444166 ]]


In [189]:
b, a = signal.butter(2, 2*400/1000, 'low')

In [190]:
print(b,a)

[0.63894553 1.27789105 0.63894553] [1.        1.1429805 0.4128016]


In [191]:
audio_lowpass=butterworth_low_pass_filter(audio_data,2,400,audio_Fs)

(29500,)


In [192]:
print(audio_lowpass)

[ 0.06471533  0.42868168  0.31707522 ... -0.06511216 -0.13562029
 -0.1444166 ]


In [193]:
#not tested
def butterworth_low_pass_filter(original_signal,order,cutoff,sampling_frequency, figure=False):
    from scipy import signal
    b, a = signal.butter(order, 2*cutoff/sampling_frequency, 'low')
    low_pass_filtered_signal = signal.filtfilt(b, a, np.squeeze(original_signal),padlen=3*max(len(a), len(b))-3)
    return low_pass_filtered_signal

#not tested
def butterworth_high_pass_filter(original_signal,order,cutoff,sampling_frequency, figures=False):
    from scipy import signal
    b, a = signal.butter(order, 2*cutoff/sampling_frequency, 'high')
    high_pass_filtered_signal = signal.filtfilt(b, a,np.squeeze(original_signal),padlen=3*max(len(a), len(b))-3)
    return high_pass_filtered_signal

#not tested
def schmidt_spike_removal(original_signal,fs,figures=False):
    from statistics import median
    import numpy as np 
    #Find the window size
    windowsize = round(fs/2)
    #Find any samples outside of a integer number of windows
    trailingsamples = len(original_signal)%windowsize
    #Reshape the signal into a number of windows

    sampleframes=[original_signal[i:(i+windowsize)] for i in range((len(original_signal)-(trailingsamples))//windowsize)]
    #Find the MAAs
    MAAs = [ max([abs(number) for number in individual_frame]) for individual_frame in sampleframes]

    #While there are still samples greater than 3* the median value of the
    #MAAs, then remove those spikes:
    while any(MAA >median(MAAs)*3 for MAA in MAAs):
        #Find the window with the max MAA
        val = max(MAAs)
        window_num=MAAs.index(val)
        if len(window_num)>1:
            window_num = window_num[0]
        #Find the postion of the spike within that window
        abssampleframe=[abs(number) for number in sampleframes[window_num]]
        spike_position=abssampleframe.index(max(abssampleframe))
        if len(spike_position)>1:
            spike_position = spike_position[0]
        #Finding zero crossings (where there may not be actual 0 values, just a change from positive to negative)
        selected_window=np.array(sampleframes[window_num])
        zero_crossings=np.abs(np.diff(np.sign(selected_window)))
        zero_crossings.append(0)
        zero_crossings=(zero_crossings>1).astype(int)
        #Find the start of the spike, finding the last zero crossing before
        #spike position. If that is empty, take the start of the window:
        zero_crossings_list=zero_crossings.tolist()
        start_nonzeros=[i for i, e in enumerate(zero_crossings_list[:spike_position]) if e != 0]
        if not start_nonzeros:
            spike_start=start_nonzeros[-1]
        else:
            spike_start=0

        #Find the end of the spike, finding the first zero crossing after
        #spike position. If that is empty, take the end of the window:
        zero_crossings_list[:spike_position] = 0
        after_nonzeros==[i for i, e in enumerate(zero_crossings_list) if e != 0]
        if not start_nonzeros:
            spike_end=start_nonzeros[-1]
        else:
            spike_end=windowsize

        #Set to Zero
        sampleframes[window_num][spike_start:spike_end] = 0.0001

        #Recaclulate MAAs
        MAAs = [ max([abs(number) for number in individual_frame]) for individual_frame in sampleframes]

    despiked_signal = np.array(sampleframes)

    # Add the trailing samples back to the signal:
    despiked_signal=np.append(despiked_signal,np.array(original_signal[despiked_signal.size:-1]))

    return despiked_signal


In [194]:
audio_data=audio_data.T
audio_data = butterworth_low_pass_filter(audio_data,2,400,audio_Fs)
audio_data = butterworth_high_pass_filter(audio_data,2,25,audio_Fs)
# Spike removal from the original paper:
#audio_data = schmidt_spike_removal(audio_data,audio_Fs)

In [195]:
print(audio_data)

[ 0.01266002  0.3847521   0.28196736 ... -0.01401128 -0.02980423
  0.02383006]


In [196]:
#schimidt_spike removal not working properly
audio_data = schmidt_spike_removal(audio_data,audio_Fs)

In [216]:
audio_data

array([ 0.01266002,  0.3847521 ,  0.28196736, ...,  0.42161233,
       -0.01321982, -0.21755915])

In [208]:
def Homomorphic_Envelope_with_Hilbert(input_signal, samplingFrequency,lpf_frequency=8,figures=False):
    #8Hz, 1st order, Butterworth LPF
    from scipy import signal
    import numpy as np
    B_low,A_low = signal.butter(1,2*lpf_frequency/samplingFrequency,'low')
    hilbert_coeff=np.array(signal.hilbert(input_signal))
    homomorphic_envelope = np.exp(signal.filtfilt(B_low,A_low,np.squeeze(np.log(np.abs(hilbert_coeff))),padlen=3*max(len(a), len(b))-3))
    #Remove spurious spikes in first sample:
    homomorphic_envelope[0] = homomorphic_envelope[1]

    return homomorphic_envelope

In [209]:
homomorphic_envelope = Homomorphic_Envelope_with_Hilbert(audio_data, audio_Fs)

In [210]:
homomorphic_envelope

array([0.41044909, 0.41044909, 0.40104687, ..., 0.36977305, 0.36213315,
       0.35464342])

In [206]:
B_low,A_low = signal.butter(1,2*8/1000,'low')

In [207]:
print(B_low,A_low)

[0.02452161 0.02452161] [ 1.         -0.95095678]


In [213]:
audio_data.shape[-1]

29500

In [215]:
signal.hilbert(audio_data)

array([ 0.01266002-0.30094678j,  0.3847521 -0.12993294j,
        0.28196736+0.17075956j, ...,  0.42161233+0.40198849j,
       -0.01321982+0.37196772j, -0.21755915+0.03602769j])